<h1 style="color:red;"><center>TPS_May21 XGBOOST+LightAutoML</center></h1>


<h2 style="color:blue;"> Importing  Required Libraries</h2>


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train=pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test=pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
submission=pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print('Shape of train Data: ',train.shape )
print('Shape of test Data: ',test.shape )
print('Shape of sample_sub Data: ',submission.shape )

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.isnull().sum()

In [ ]:
# Dropping ID Column

train=train.drop(['id'],axis=1)
test=test.drop(['id'],axis=1)

In [ ]:
train['target'].value_counts().sort_index(ascending=True)

In [ ]:
plt.figure(figsize=(10,7))
sns.countplot(x=train['target'])

In [ ]:
train['Target']=train['target'].map({'Class_1':0,'Class_2':1,'Class_3':2,'Class_4':3})
train['Target'].unique()

In [ ]:
!pip install dataprep

In [ ]:
from dataprep.eda import plot, plot_correlation, create_report, plot_missing
plot(train)

### **Some of the features have negative values**

### Lets check for Outlier's

In [ ]:
# Checking outlier's in Train Data
plt.figure(figsize=(18,25))
sns.boxplot(data=train, orient="h");

In [ ]:
# Checking outlier's in Test Data
plt.figure(figsize=(18,25))
sns.boxplot(data=test, orient="h");

### Lets Check Correlations between features

In [ ]:
# Pearson Correlation
plt.figure(figsize=(18,10))
sns.heatmap(train.corr(method='pearson'), cbar=False, annot=True, fmt='.1f', linewidth=0.2, cmap='coolwarm');

In [ ]:
# Spearman Correlation
plt.figure(figsize=(18,12))
sns.heatmap(train.corr(method='spearman'), cbar=False, annot=True, fmt='.1f', linewidth=0.2, cmap='coolwarm');

In [ ]:
fig, ax = plt.subplots(figsize=(18, 12))
corr = train.corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool))
ax.text(-1.1, -0.7, 'Correlation between the Features', fontsize=20, fontweight='bold', fontfamily='serif')
sns.heatmap(corr, mask=mask, annot=False, fmt='.2f', linewidth=0.2, cbar=True, cmap='coolwarm');

In [ ]:
# kendall
fig, ax = plt.subplots(1, 3, figsize=(17 , 5))

feature_lst = ['feature_0', 'feature_1', 'feature_2','feature_3','feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9']

corr = train[feature_lst].corr()

mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True


for idx, method in enumerate(['pearson', 'kendall', 'spearman']):
    sns.heatmap(train[feature_lst].corr(method=method), ax=ax[idx],
            square=True, annot=True, fmt='.1f', center=0, linewidth=2,
            cbar=False, cmap=sns.diverging_palette(240, 10, as_cmap=True),
            mask=mask
           ) 
    ax[idx].set_title(f'{method.capitalize()} Correlation', loc='left', fontweight='bold')     

plt.show()

In [ ]:
# Correlation between Independent and Dependent Values
train.corr()['Target'].sort_values(ascending=False)

In [ ]:
a = train.drop(['target','Target'], axis=1)
a.corrwith(train['Target']).plot(kind='bar', figsize=(18,11), color=['salmon'])
plt.title('Correlation Between target and Independant features')
plt.xticks(size=15)
plt.yticks(size=15)
plt.show()

In [ ]:
X = train.iloc[:,:-2]
X.head()

In [ ]:
y = train['target']
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

<h1 style="color:blue;"> <center> Model Building </center></h1>


<h2 style="color:blue;"> XGBOOST</h2>


In [ ]:
from xgboost import XGBClassifier, plot_importance

In [ ]:
model = XGBClassifier(random_state=42, use_label_encoder=True)
model.fit(X, y)

In [ ]:
y_pred_xgb = model.predict_proba(test)

In [ ]:
submission_xgb = pd.DataFrame(y_pred_xgb, columns=['Class_1','Class_2','Class_3','Class_4'])
submission_xgb['id'] = submission['id']
submission_xgb


 
<h2 style="color:blue;"> LightAutoML</h2>


In [ ]:
pip install -U lightautoml

In [ ]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from sklearn.metrics import log_loss

In [ ]:
N_THREADS = 4 
N_FOLDS = 5 
RANDOM_STATE = 2021 
TEST_SIZE = 0.2 
TIMEOUT = 60 * 60 

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['target'] = le.fit_transform(train['target'])

In [ ]:
%%time

automl = TabularUtilizedAutoML(task = Task('multiclass',), 
                               timeout = TIMEOUT,
                               cpu_limit = N_THREADS,
                               verbose=0,
                               reader_params = {'n_jobs': N_THREADS},
)


In [ ]:
"""target_column = 'target'

roles = {
    'target': target_column,
    'drop': ['id'],
}

lightml_pred = automl.fit_predict(train.iloc[:,:-1], roles = roles)
print('lightml_pred:\n{}\nShape = {}'.format(lightml_pred[:10], lightml_pred.shape))"""

In [ ]:
"""%%time

test_pred = automl.predict(test)
print('Prediction for test set:\n{}\nShape = {}'.format(test_pred[:5], test_pred.shape))"""

In [ ]:
"""submision.iloc[:, 1:] = test_pred.data
subission.to_csv('output.csv', index = False)"""